In [2]:
ls data

 Volume in drive G is JKomSchool
 Volume Serial Number is 58A9-2428

 Directory of G:\belmont\spring_2020\stat_modeling\mth2895\data

02/11/2020  01:55 PM    <DIR>          .
02/11/2020  01:55 PM    <DIR>          ..
01/14/2020  02:49 PM             4,012 Advertising.csv
01/14/2020  02:49 PM            19,004 auto.csv
01/14/2020  02:49 PM            40,563 boston.csv
01/14/2020  02:49 PM         1,013,001 caravan.csv
01/14/2020  02:49 PM            16,628 carseats.csv
01/14/2020  02:49 PM            75,701 college.csv
01/14/2020  02:49 PM            21,922 Credit.csv
01/14/2020  02:49 PM           404,710 default.csv
02/06/2020  02:20 PM            59,175 employees.csv
01/14/2020  02:49 PM            27,687 hitters.csv
01/14/2020  02:49 PM           619,618 khan_xtest.csv
01/14/2020  02:49 PM         1,512,925 khan_xtrain.csv
01/14/2020  02:49 PM               138 khan_ytest.csv
01/14/2020  02:49 PM               439 khan_ytrain.csv
01/14/2020  02:49 PM         3,118,789 nci60_data.csv

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.datasets import load_digits


digits = load_digits()

### Using the Default dataset

Preprocess the `default` dataset

In [4]:
df = pd.read_csv('data/default.csv')

# changes binary categorical columns to binary
df['student_yes'] = np.where(df['student'] == 'Yes', 1, 0)
df['default_yes'] = np.where(df['default'] == 'Yes', 1, 0)
#drop the old columns
df.drop(['student'], axis=1, inplace=True)
df.drop(['default'], axis=1, inplace=True)
# normalize/scale default balance column
df['balance'] = (df['balance'] - df['balance'].mean()) / (df['balance'].max() - df['balance'].min())
df.head()

,balance,income,student_yes,default_yes
0,-0.039878,44361.625074,0,0
1,-0.006855,12106.134700,1,0
2,0.089731,31767.138947,0,0
3,-0.115330,35704.493935,0,0
4,-0.018731,38463.495879,0,0


Separate the X and y for `sklearn`, using `income` and `balance` for now

In [5]:
# seperate x and y for sklearn
# y is dependant value
y = df[df.columns[-1]] # last column
y.shape # only one list
# X is features
X = df[df.columns[0:3]]
X.shape # size is amount o features
X.head()

,balance,income
0,-0.039878,44361.625074
1,-0.006855,12106.134700
2,0.089731,31767.138947
3,-0.115330,35704.493935
4,-0.018731,38463.495879


Use `sklearn` to fit a logistic regression model using only the training set and print the parameters (*hint*: use C=100000, tol=.0000001 values will help you create a decent model)

In [6]:
# creating logistic regression model with just training set (whole set currently)
lr = LogisticRegression(C=100000, tol=.0000001)
lr.fit(X, y)
lr.score(X, y)
# print out beta, coefficients
# lr.coef_

C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9738

Use `statsmodels` to fit a logistic regression model

In [7]:
# first param in '' is the depandant variable, then ~ followed by the 
# features
results = smf.logit('default_yes ~ income + balance', data = df).fit()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


In [8]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_yes   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Thu, 20 Feb 2020   Pseudo R-squ.:                  0.4594
Time:                        16:29:16   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.8230      0.271    -25.195      0.000      -7.354      -6.292
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance       14.9892      0.604     24.835      0.000      13.806      16.172
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Get the error rate for the `sklearn` model without validation set. *Hint*: the `predict` method will be helpful 

In [9]:
from sklearn.metrics import accuracy_score
lr_y_pred = lr.predict(X)
lr_a_score = accuracy_score(y, lr_y_pred)
lr_error_rate = 1 - lr_a_score
lr_error_rate

0.0262

Get the error rate for the `statsmodel` model without validation set. *Hint*: the `predict` method will be helpful 

In [10]:
(((results.predict(X)) > .5) * 1 == y).mean()

#sm_y_pred = results.predict(X)
#sm_a_score = accuracy_score(y, sm_y_pred)

0.9737

Create a new Logistic Regression model using the train-test split approach using both `sklearn` and `statsmodel`

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=3)
X_train.shape, X_valid.shape, X_test.shape

((5625, 2), (1875, 2), (2500, 2))

Join the X and y columns so that we have a complete dataset for `statsmodel`

In [12]:
df_train = X_train.join(y_train) # join the train set
df_valid = X_valid.join(y_valid) # join the valid set
df_test = X_test.join(y_test)

Fit a logistic regression model using `statsmodel` and print the results

In [13]:
results = smf.logit('default_yes ~ income + balance', data = df_train).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.080366
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_yes   No. Observations:                 5625
Model:                          Logit   Df Residuals:                     5622
Method:                           MLE   Df Model:                            2
Date:                Thu, 20 Feb 2020   Pseudo R-squ.:                  0.4557
Time:                        16:29:16   Log-Likelihood:                -452.06
converged:                       True   LL-Null:                       -830.47
Covariance Type:            nonrobust   LLR p-value:                4.574e-165
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.9103      0.361    -19.167      0.000      -7.617      -6.204
income      2.263e-05   6.51e-06      3.475      0.001    9.87e-06    3.54e-05
balance       15.0523      0.810     18.575      0.000      13.464      16.641
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Get the error rates for the `sklearn` and `statsmodel` models using the test set and compare the errors with the above models fitted without a validation set. *Hint*: the `predict` method will be helpful. 

In [14]:
((results.predict(X) > .5) * 1 == y).mean(), (lr.predict(X) == y).mean()

(0.9738, 0.9738)

Validation error of only .022

Use a for loop to repeat the above process for 3 different train/test splits

In [15]:
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    lr = LogisticRegression(C=100000, tol=.0000001)
    lr.fit(X_train, y_train)
    # print(lr.score(X_test, y_test))
    
    X_train_sm = X_train.join(y_train)
    result = smf.logit('default_yes ~ income + balance', data=X_train_sm).fit()
    print(((result.predict(X) > .5) * 1 == y).mean(), (lr.predict(X) == y).mean())

Optimization terminated successfully.
         Current function value: 0.080125
         Iterations 10
0.9735 0.9735
Optimization terminated successfully.
         Current function value: 0.077937
         Iterations 10
0.9737 0.9737
Optimization terminated successfully.
         Current function value: 0.079620
         Iterations 10
0.9737 0.9738


C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Repeat the above step with student included in the model


In [17]:
# seperate x and y for sklearn
# y is dependant value
y = df[df.columns[-1]] # last column
y.shape # only one list
# X is features
X = df[df.columns[0:3]]
X.shape # size is amount o features
X.head()

,balance,income,student_yes
0,-0.039878,44361.625074,0
1,-0.006855,12106.134700,1
2,0.089731,31767.138947,0
3,-0.115330,35704.493935,0
4,-0.018731,38463.495879,0


In [18]:
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(X, y) ## x set needs to change dependant on independant variables, we're adding student_yes here so it needs to be added to x as well
    lr = LogisticRegression(C=100000, tol=.0000001)
    lr.fit(X_train, y_train)
    # print(lr.score(X_test, y_test))
    
    X_train_sm = X_train.join(y_train)
    result = smf.logit('default_yes ~ income + balance + student_yes', data=X_train_sm).fit()
    print(((result.predict(X) > .5) * 1 == y).mean(), (lr.predict(X) == y).mean())

Optimization terminated successfully.
         Current function value: 0.079958
         Iterations 10
0.9733 0.9733
Optimization terminated successfully.
         Current function value: 0.075589
         Iterations 10
0.9735 0.9732
Optimization terminated successfully.
         Current function value: 0.078721
         Iterations 10
0.9734 0.9734


C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The error rates should be very similar

### Use the [digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) dataset from sklearn

Split the digits into train and test sets. *Hint*: the features are stored in digits.data and the target values are in digits.target

In [19]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)
X_train.shape
y_train.shape

(1257,)

Fit a logistic regression model using the train/test split in the previous step

In [20]:
lr = LogisticRegression(C=100000, tol=.0000001)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9296296296296296

For now, let's fit two more models using SVC and RandomForest

In [21]:
smv = SVC()
smv.fit(X_train, y_train)
smv.score(X_test, y_test)

# measure accuracy and area under the ROC curve
# precision as well
# recaulc?

C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.362962962962963

In [22]:
rf = RandomForestClassifier(n_estimators =40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.975925925925926

Use k-fold cross validation with [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

In [23]:
kf = KFold(n_splits=3)

For starters, pass in a small, random dataframe and observe what sets are produced from KFold

In [24]:
for train_index, test_index in kf.split([1, 2, 3, 4, 5, 6, 7, 8, 9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


Create a method that generalizes the model creation process (*hint*: your method should have 5 parameters)

In [25]:
# model is whichevber model you are using
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

Test our method

In [26]:
get_score(SVC(), X_train, X_test, y_train, y_test)

C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.362962962962963

Compare the 3 models using k-fold CV

Shortcut: `cross_val_score` from `model_selection` will do exactly what we did in the above cell!

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
cross_val_score(LogisticRegression(), digits.data, digits.target, cv=3)

C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\jkom8\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default so

array([0.89534884, 0.94991653, 0.90939597])